In [1]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_WedgeFlow_OptProbStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 6; Grid Count = 6; Path = \\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_OptProbStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 21:34:30Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:57 PM	a8e8a6b5...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:43 PM	01b113de...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:35:22 PM	13900961...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:35:09 PM	9639e527...
#4: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:16 PM	ef70b687...
#5: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:30 PM	0f400bef...


In [5]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

a8e8a6b5-3243-4cf6-8381-3563d669adc1

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the WedgeFlow ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=2$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [8]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses=sessions.Where(s => (s.SuccessfulTermination )); // the one plotted in the publication
ses


#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:57 PM	a8e8a6b5...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:43 PM	01b113de...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:35:22 PM	13900961...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:35:09 PM	9639e527...
#4: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:16 PM	ef70b687...
#5: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 9:34:30 PM	0f400bef...


# A Helper Function to obtain different plot formats

In [9]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [10]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M489.0,57.1 L542.4,57.1 M53.9,50.9 L75.8,52.8 L97.7,53.7 L119.7,53.9 L141.6,103.0 L163.5,104.7
 L185.4,112.4 L207.4,116.1 L229.3,120.6 L251.2,121.9 L273.1,124.5 L295.1,127.3 L317.0,133.6 L338.9,141.6
 L360.8,144.9 L382.8,147.5 L404.7,157.8 L426.6,168.9 L448.5,198.1 L470.5,222.2 L492.4,363.0 L514.3,363.0
 L536.2,438.9 L558.2,438.9 L580.1,466.7 L602.0,466.3 L623.9,445.1 L645.8,458.0 L667.8,526.0 L689.7,516.7
 L711.6,516.7 L733.5,516.7 L755.5,516.7 L777.4,516.7 L799.3,517.2 L821.2,522.0 L843.2,521.0 L865.1,530.7
 L887.0,533.6 L908.9,531.7 L930.9,531.2 L952.8,531.2 L974.7,531.2 L996.6,531.2 L1018.6,531.2 L1040.5,531.2
 L1062.4,530.5 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M489.0,81.1 L542.4,81.1 M53.9,50.9 L75.8,52.8 L97.7,53.7 L119.7,53.9 L141.6,63.4 L163.5,65.2
 L185.4,74.8 L207.4,78.7 L229.3,88.2 L251.2,90.1 L273.1,120.8 L295.1,125.7 L317.0,132.8 L338.9,139.4
 L360.8,147.0 L382.8,147.3 L404.7,153.2 L426.6,199.9 L448.5,288.8 L470.5,348.0 L492.4,354.0 L514.3,354.0
 L536.2,371.4 L558.2,364.1 L580.1,364.1 L602.0,386.6 L623.9,386.6 L645.8,380.4 L667.8,383.8 L689.7,384.0
 L711.6,384.1 L733.5,384.1 L755.5,384.1 L777.4,380.7 L799.3,375.1 L821.2,375.1 L843.2,375.0 L865.1,375.0
 L887.0,375.2 L908.9,375.2 L930.9,375.2 L952.8,375.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M489.0,105.1 L542.4,105.1 M53.9,50.9 L75.8,54.9 L97.7,56.7 L119.7,57.2 L141.6,59.0 L163.5,60.9
 L185.4,64.8 L207.4,68.8 L229.3,105.9 L251.2,108.0 L273.1,112.7 L295.1,116.9 L317.0,127.8 L338.9,139.4
 L360.8,144.3 L382.8,151.8 L404.7,162.7 L426.6,202.0 L448.5,244.8 L470.5,328.2 L492.4,385.1 L514.3,397.3
 L536.2,404.8 L558.2,411.5 L580.1,418.0 L602.0,424.2 L623.9,430.3 L645.8,436.3 L667.8,442.3 L689.7,448.3
 L711.6,454.2 L733.5,460.2 L755.5,466.1 L777.4,472.0 L799.3,477.9 L821.2,470.5 L843.2,473.9 L865.1,477.3
 L887.0,480.6 L908.9,484.0 L930.9,485.6 L952.8,486.3 L974.7,486.3 L996.6,486.3 L1018.6,486.3 L1040.5,486.3
 L1062.4,486.3 L1084.3,486.3 L1106.3,486.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M489.0,153.1 L542.4,153.1 M53.9,50.9 L75.8,52.8 L97.7,53.2 L119.7,54.1 L141.6,63.6 L163.5,67.6
 L185.4,77.1 L207.4,78.0 L229.3,87.6 L251.2,91.5 L273.1,123.9 L295.1,127.7 L317.0,129.4 L338.9,135.6
 L360.8,137.4 L382.8,143.4 L40

## Residual History

In [13]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    lts.ResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M0.2,57.1 L53.6,57.1 M53.9,53.1 L67.8,54.5 L81.7,55.2 L95.7,55.4 L109.6,63.6 L123.5,82.4
 L137.4,85.8 L151.4,88.6 L165.3,97.4 L179.2,98.3 L193.1,101.4 L207.1,100.4 L221.0,94.7 L234.9,93.2
 L248.8,106.9 L262.8,108.7 L276.7,117.2 L290.6,122.6 L304.5,141.4 L318.5,189.4 L332.4,290.1 L346.3,290.1
 L360.2,328.6 L374.2,328.6 L388.1,344.5 L402.0,357.3 L415.9,341.5 L429.8,379.6 L443.8,391.7 L457.7,401.7
 L471.6,401.7 L485.5,401.7 L499.5,401.7 L513.4,401.7 L527.3,401.9 L541.2,401.6 L555.2,401.3 L569.1,413.3
 L583.0,413.3 L596.9,422.8 L610.9,420.7 L624.8,420.8 L638.7,420.8 L652.6,420.8 L666.6,420.8 L680.5,420.8
 L694.4,420.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M0.2,81.1 L53.6,81.1 M53.9,54.8 L67.8,56.3 L81.7,57.0 L95.7,57.0 L109.6,54.4 L123.5,65.8
 L137.4,67.5 L151.4,68.7 L165.3,70.5 L179.2,71.6 L193.1,73.5 L207.1,75.0 L221.0,79.9 L234.9,81.5
 L248.8,86.8 L262.8,89.0 L276.7,95.5 L290.6,102.3 L304.5,130.9 L318.5,178.0 L332.4,254.3 L346.3,254.3
 L360.2,316.8 L374.2,316.8 L388.1,332.1 L402.0,344.2 L415.9,352.3 L429.8,370.6 L443.8,377.1 L457.7,388.5
 L471.6,388.5 L485.5,388.5 L499.5,388.5 L513.4,388.5 L527.3,388.7 L541.2,397.8 L555.2,399.8 L569.1,398.5
 L583.0,397.7 L596.9,406.9 L610.9,406.9 L624.8,406.9 L638.7,406.9 L652.6,406.9 L666.6,406.9 L680.5,406.9
 L694.4,406.9 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M0.2,105.1 L53.6,105.1 M53.9,53.1 L67.8,54.5 L81.7,55.2 L95.7,55.4 L109.6,62.5 L123.5,64.5
 L137.4,71.5 L151.4,74.4 L165.3,81.6 L179.2,82.9 L193.1,96.0 L207.1,90.3 L221.0,89.5 L234.9,91.7
 L248.8,94.2 L262.8,106.8 L276.7,116.2 L290.6,146.8 L304.5,201.9 L318.5,278.9 L332.4,281.0 L346.3,281.0
 L360.2,285.5 L374.2,288.3 L388.1,288.2 L402.0,300.5 L415.9,300.5 L429.8,302.0 L443.8,304.6 L457.7,304.8
 L471.6,304.8 L485.5,304.8 L499.5,304.8 L513.4,305.1 L527.3,305.4 L541.2,305.4 L555.2,305.4 L569.1,305.4
 L583.0,305.5 L596.9,305.5 L610.9,305.5 L624.8,305.5 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M0.2,129.1 L53.6,129.1 M53.9,54.8 L67.8,56.4 L81.7,57.0 L95.7,57.1 L109.6,61.1 L123.5,64.6
 L137.4,66.9 L151.4,67.8 L165.3,70.0 L179.2,70.7 L193.1,72.5 L207.1,74.1 L221.0,77.4 L234.9,78.3
 L248.8,83.1 L262.8,90.3 L276.7,99.6 L290.6,133.5 L304.5,188.0 L318.5,252.7 L332.4,258.5 L346.3,258.5
 L360.2,272.7 L374.2,273.4 L388.1,273.4 L402.0,279.0 L415.9,279.0 L429.8,278.2 L443.8,278.1 L457.7,278.1
 L471.6,278.1 L485.5,278.1 L499.5,278.1 L51

### Plot ShadowFields

In [12]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm;
